In [1]:
import os
import collections
import csv
import re #Regular Expressions
import operator # Majorly for sorting tf-idf dictionary on basis of values
import nltk
from nltk.corpus import stopwords # For removing stop words, use ntlk.download() to completely install this package
import pandas as pd
import numpy as np
from sklearn import model_selection as cv
from sklearn import naive_bayes
from sklearn.metrics import classification_report, confusion_matrix
import math

# Parameters to customize according to use case

In [2]:
base_dir = "/home/aakash/Drive/Dropbox/ML/data/20_newsgroups/" #Set  this according to your machine
class_names = os.listdir(base_dir)

In [60]:
#Use small numbers for first time, articles =  100, classes = 2 , features = 100 
max_articles_of_each_class = 1000
no_of_classes = 5
no_of_features = 5000

# Caclulating tf-idf metric for articles of selected classes

In [61]:
def preprocess_file_text(file):
    text = file.read().lower() # Converting all to lowercase as lowercase and uppercase words should be considered same word
    text = re.sub('[^A-Za-z ]+', '', text) # Removing non-aplha characters
    text = re.sub('\s+', ' ', text)  # Condense all whitespace
    return text

In [62]:
features = set()
no_of_features_per_class =  no_of_features//no_of_classes
# tf = {}
# idf = {}
selected_classes = []
articles_read = {} # To keep track of which all articles were read while extracting features
for i in range(0,len(class_names)):
    tf = {}
    idf = {}
    if(i >= no_of_classes):
        break
    current_class = class_names[i]
    
    # 2 updates for tracking
    selected_classes.append(current_class)
    articles_read[current_class] = []
    
    class_dir = base_dir + current_class
    all_articles = os.listdir(class_dir)
    for j in range(0,len(all_articles)):
        if(j >= max_articles_of_each_class):
            break
        current_file = class_dir + "/" + all_articles[j]
        articles_read[current_class].append(all_articles[j]) 
        file = open(current_file, encoding = "ISO-8859-1")
        text = preprocess_file_text(file)
        file.close() # Always close a file after using it to free up system resources
        file_words = text.split()
        
        # Updating term-frequency dictionary
        word_count = collections.Counter(file_words)
        for word,freq in word_count.items():
            if(word in tf):
                tf[word] = tf[word] + freq
            else:
                tf[word] = freq
        
        #Updating (inverse document frequency) dictionary
        word_set = set(file_words)
        for word in word_set:
            if(word in idf):
                idf[word] = idf[word] + 1
            else:
                idf[word] = 1
    
    #Taking equal no. of features from each class
    tf_by_idf = {}
    for key in tf.keys():
        tf_by_idf[key] = tf[key]/idf[key]
    for stop_word in stopwords.words("english"):
        if(stop_word in tf_by_idf.keys()):
            tf_by_idf.pop(stop_word)
    tf_by_idf = sorted(tf_by_idf.items(), key=operator.itemgetter(1))
    tf_by_idf.reverse()
    for i in range(0,no_of_features_per_class):
        features.add(tf_by_idf[i][0])

# Creating Dataframe by reading the articles again on the basis of selected features

In [63]:
# It takes time,so be patient (Reduce feature count to reduce this time but score will be affected accordingly)

columns = list(features)
total_articles_to_process = 0
for class_name in articles_read.keys():
    total_articles_to_process += len(articles_read[class_name])
data = []

articles_processed = 0
for current_class in selected_classes:
    class_dir = base_dir + current_class
    for article in articles_read[current_class]:
        articles_processed += 1
        if(articles_processed%500 == 0):
            print(articles_processed,"articles are processed out of",total_articles_to_process,"articles")
        current_file = class_dir + "/" + article
        file = open(current_file, encoding = "ISO-8859-1")
        text = preprocess_file_text(file)
        file.close() # Always close a file after using it to free up system resources
        file_words = text.split()
        
        word_count = collections.Counter(file_words)
        training_data = [0]*(len(columns) + 1) # +1 because last column is of output class
        for i in range(0,len(columns)): 
            feature = columns[i]
            if(feature in word_count.keys()):
                training_data[i] = word_count[feature]
        training_data[-1] = current_class
        data.append(training_data)

columns.append("class")
df = pd.DataFrame(data,columns=columns)

500 articles are processed out of 5000 articles
1000 articles are processed out of 5000 articles
1500 articles are processed out of 5000 articles
2000 articles are processed out of 5000 articles
2500 articles are processed out of 5000 articles
3000 articles are processed out of 5000 articles
3500 articles are processed out of 5000 articles
4000 articles are processed out of 5000 articles
4500 articles are processed out of 5000 articles
5000 articles are processed out of 5000 articles


In [64]:
X = df.values[:,:-1]
Y = df.values[:,-1]

In [65]:
X_train,X_test,Y_train,Y_test = cv.train_test_split(X,Y,test_size=0.05,random_state=0)

# Using Inbuilt Multinomial Naive Bayes

In [66]:
gnb = naive_bayes.GaussianNB()
gnb.fit(X_train,Y_train)
Y_pred = gnb.predict(X_test)
print("Size of Y_test = ",len(Y_test)," and wrong results = ",(Y_pred != Y_test).sum())
print("Score is ",gnb.score(X_test,Y_test))
#print(gnb.predict_proba(X_test[Y_test != Y_pred]))

Size of Y_test =  250  and wrong results =  61
Score is  0.756


# Using my implementation (Incomplete)

In [67]:
def calculatePriorProbabilities(Y):
    classes = set(Y)
    result = {}
    for i in classes:
        result[i] = (len(Y[Y==i])/len(Y))
    return result

In [68]:
def gaussianProbability(column,x):
    mean = column.mean()
    variance = 1
    power = -((x - mean)**2)/(2*variance)
    prob = (np.exp(power))/np.sqrt(2*math.pi*variance)
    return prob

In [69]:
def naiveProbability(priorProbability,current_class,X_train,Y_train,X_test_sample):
    result = np.log10(priorProbability)
    #Modifying X_train for current class only
    class_samples = (Y_train == current_class)
    Y_train = Y_train[class_samples]
    X_train = X_train[class_samples]
    for i in range(0,X_train.shape[-1]):
        gp = gaussianProbability(X_train[:,i],X_test_sample[i])# len(X_train[X_train[:,i]==X_test_sample[i]])
        result = result + np.log10(gp)
    return result

In [70]:
#For continous data using Gaussian Probability
#Using Log with base 10 of probabilitites
def naiveBayesPredict(X_train,Y_train,X_test,priorProbabilities={}):
    classes = set(Y_train)
    test_samples = X_test.shape[0]
    y_pred = [0] * test_samples
    
    #Assuming this condition is sufficient to check if this variable was passed or not
    if (len(priorProbabilities) == 0) :
        priorProbabilities = calculatePriorProbabilities(Y_train)
    #print(priorProbabilities)
    for i in range(0,test_samples):
        probabilities = {}
        for current_class in classes:
            probabilities[current_class] = naiveProbability(priorProbabilities[current_class],current_class,X_train,Y_train,X_test[i,:])
        #print("For sample i = ",i," probabilities are = ",probabilities)
        y_pred[i] = max(probabilities,key=probabilities.get)
    return y_pred

In [71]:
Y_pred = naiveBayesPredict(X_train,Y_train,X_test)
print("Size of Y_test = ",len(Y_test)," and wrong results = ",(Y_pred != Y_test).sum())

Size of Y_test =  250  and wrong results =  75


# Few Findings

### Algo worked better overall with class wise distribution of features
### Use log of probabilities, otherwise most of probabilities will turn out to be so small that it would eventually become zero and can't be compared for different classes.
### Issue occurs if order of size of dataframe becomes more than 10^7. So, how to deal with 20 classes with 1000 articles each if we consider atleast 1000 featues, the size will be of order 2*10^7. Algo will perform much better with around 10^4 feature count. 